<a href="https://colab.research.google.com/github/asefycom/Python-with-AI/blob/main/3rd-week/L07/L07_create_itinerary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# هفته سوم - درس ۷: ساخت برنامه سفر به چند شهر

با چیزهایی که تا حالا یاد گرفتی تو این درس بناست که برنامه سفر به دور دنیا برای تعطیلات بسازی.

## پیشنیازها

In [ ]:
# کلون مخزن گیت هاب  و تغییر مسیر کاری به پوشه این درس
import os

if not os.path.exists('Python-with-AI'):
    !git clone https://github.com/asefycom/Python-with-AI.git

os.chdir('Python-with-AI/3rd-week/L07')
print("Current Working Directory:", os.getcwd())

Cloning into 'Python-with-AI'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 272 (delta 63), reused 4 (delta 4), pack-reused 154 (from 1)
Receiving objects: 100% (272/272), 158.47 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Current Working Directory: /content/Python-with-AI/3rd-week/L07


In [ ]:
# ایمپورت‌های لازم
from IPython.display import Markdown, HTML
import csv
import pandas as pd

In [ ]:
# تابعی برای نمایش داده ها به صورت جدول
def display_table(data):
    # تبدیل داده ورودی به دیتافریم
    df = pd.DataFrame(data)

    # نمایش دیتافریم به شکل جدول ا‌چ‌تی‌ام‌ال
    display(HTML(df.to_html(index=False)))

In [ ]:
# نصب بسته توگدر
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
  Attempting uninstall: typer
    Found existing installation: typer 0.16.0
    Uninstalling typer-0.16.0:
      Successfully uninstalled typer-0.16.0


In [ ]:
# فراخوانی توگدر با کلیدی که ازش گرفتیم
# که به صورت امن تو محیط اجرای این دفترچه کولب کار گذاشتیم
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('TOGETHER_API_KEY'))

# تابعی برای ارسال پرامپت به مدل زبانی به واسطه توگدر و بعد چاپ جواب
def print_llm_response(prompt):
    response = client.chat.completions.create(
        model="google/gemma-3n-E4B-it",
        # model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=[
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return response.choices[0].message.content

## واکشی برنامه‌ سفر از فایل‌ سی‌اس‌وی

نخست بیا تابعی تعریف کنیم که اطلاعات فایل سی‌اس‌وی رو بخونه و به صورت یک دیکشنری به ما بده.

In [ ]:
def read_csv(file):
    f = open(file, "r")

    csv_reader = csv.DictReader(f)
    data = []
    for row in csv_reader:
        data.append(row)
    f.close()

    return data

حالا از این تابع استفاده کنیم. برنامه‌های سفر رو از فایل

 `itinerary.csv`

 بخونیم و اطلاعاتش رو که به صورت دیکشنری دریافت می‌کنیم و به صورت یک جدول نمایش بدیم.

In [ ]:
# Read the itinerary.csv file
itinerary = read_csv("itinerary.csv")

# Display the itinerary
display_table(itinerary)

Arrival,Departure,City,Country
July-01,July-08,New York,USA
July-09,July-16,Rio de Janeiro,Brazil
July-17,July-24,Cape Town,South Africa
July-25,August-01,Istanbul,Turkey
August-02,August-09,Paris,France
August-10,August-17,Tokyo,Japan
August-18,August-25,Sydney,Australia


## واکشی اطلاعات رستوران‌ها از یادداشت‌های غذایی

حالا تابعی بسازیم که محتوای فایل متنی رو می‌خونه و به صورت یک استرینگ بلند برمی‌گردونه.

In [ ]:
# این تابع رو «رید جورنال» نام گذاشتیم
def read_journal(journal_file):
    f = open(journal_file, "r")
    journal = f.read()
    f.close()

    # بازگشت محتوای یادداشت
    return journal

حالا برای نمونه از این تابع استفاده کنیم و یادداشت شهر سیدنی رو بخونیم.

In [ ]:
journal = read_journal("sydney.txt")

print(journal)

My culinary adventure began at Saint Peter, a renowned seafood restaurant in Paddington. This place is a temple to Australian seafood, and the "Murray Cod" was a revelation. The fish, sourced from the Murray River, was perfectly cooked, with a crispy skin and tender, flaky flesh. It was served with a simple yet flavorful accompaniment of seasonal vegetables, allowing the quality of the fish to shine. The restaurant's dedication to sustainability and nose-to-tail seafood cooking added an educational aspect to the delicious meal.

Next, I visited Billy Kwong in Potts Point, where celebrated chef Kylie Kwong puts a unique spin on modern Australian cuisine using native ingredients. The standout dish here was the "Crispy Skin Duck with Davidson’s Plum Sauce." The duck was cooked to perfection, with a rich, flavorful meat and delightfully crispy skin, complemented by the tart and slightly sweet Davidson’s plum sauce. This dish was a perfect example of how traditional recipes can be elevated 

و حالا ... یک پرامپت (سپارش) بنویسیم که اطلاعات رستوران‌ها و غذای مخصوص هر کدوم رو استخراج و به فرمت سی‌اس‌وی ذخیره کنه.

In [ ]:
# Write the prompt
prompt = f"""Please extract a comprehensive list of the restaurants
and their respective specialties mentioned in the following journal entry.
Ensure that each restaurant name is accurately identified and listed.
Provide your answer in CSV format, ready to save.
Exclude the "```csv" declaration, don't add spaces after the comma, include column headers.

Format:
Restaurant, Specialty
Res_1, Sp_1
...

Journal entry:
{journal}
"""

# Print the prompt
print(print_llm_response(prompt))

Restaurant,Specialty
Saint Peter,Australian Seafood, Murray Cod
Billy Kwong,Modern Australian Cuisine, Crispy Skin Duck with Davidson’s Plum Sauce
The Lord Nelson Brewery Hotel,Classic Australian Fare, Roast Lamb
Vic's Meat Market,Australian Barbecue, BBQ Beef Brisket
Bennelong,Australian Produce, Sydney Rock Oysters



از مدل زبانی بخواه تابعی برای ذخیره داده‌های سی اس وی به صورت فایل به زبان پایتون تولید کنه.

این پایین بنویسش.

In [ ]:
# تابع ایجاد فایل سی اس وی رو بنویس


فایل پایین یک نمونه ذخیره شده از خروجی مدل هست:

`Sydney.csv`

فایل رو بخون و به صورت جدول نمایش بده:

In [ ]:
# استفاده از تابعی که پیش‌تر تعریف کردی
sydney_restaurants = read_csv("Sydney.csv")

display_table(sydney_restaurants)

Restaurant,Specialty
Saint Peter,Murray Cod
Billy Kwong,Crispy Skin Duck with Davidson’s Plum Sauce
The Lord Nelson Brewery Hotel,Roast Lamb
Carriageworks Farmers Market,Kangaroo Pie
Vic's Meat Market,BBQ Beef Brisket
Bennelong,Sydney Rock Oysters


## ساخت برنامه سفر کامل با پیشنهادهایی برای رستوران

بیا داده‌های برنامه  سفر و یادداشت غذا رو کنار هم بگذاریم و یک برنامه کامل برای بازدید از سیدنی بسازیم.

دقت کن که اطلاعات سیدنی در ایندکس ۶ «فایل برنامه سفر» درج شده چون سیدنی هفتمین مقصد سفر هست.

In [ ]:
# انتخاب سیدنی از برنامه سفر
trip_stop = itinerary[6]

حالا همه اطلاعات برنامه سفر سیدنی رو داخل متغیرهای جداگانه ذخیره کن:

In [ ]:
city = trip_stop["City"]
country = trip_stop["Country"]
arrival = trip_stop["Arrival"]
departure = trip_stop["Departure"]
restaurants = sydney_restaurants

سرانجام همه این اطلاعات رو در یک پرامپت به مدل‌زبانی بده تا یک برنامه سفر جامع تهیه کنه.

In [ ]:
# متن پرامپت یا سپارش
prompt = f"""I will visit {city}, {country} from {arrival} to {departure}.
Create a daily itinerary with detailed activities.
Designate times for breakfast, lunch, and dinner.

I want to visit the restaurants listed in the restaurant dictionary
without repeating any place. Make sure to mention the specialty
that I should try at each of them.

Restaurant dictionary:
{restaurants}

"""

response = print_llm_response(prompt)

# چاپ پاسخ مدل به فرمت مارک‌داون
display(Markdown(response))

Okay, here is a detailed daily itinerary for your trip to Sydney from August 18th to August 25th, incorporating your restaurant preferences and aiming for a balanced mix of sightseeing, cultural experiences, and delicious food. I've included suggested times for meals and activities.  I've also made sure to avoid repeating restaurants.  Enjoy your trip!

**Important Notes:**

*   **August in Sydney:** Expect mild weather (average highs around 18°C/64°F, lows around 11°C/52°F). Pack layers, including a light jacket or sweater.
*   **Transportation:** Sydney has a good public transportation system (trains, buses, ferries). Consider getting an Opal card for easy travel.  Walking is also a great way to explore many areas.
*   **Reservations:**  I *highly* recommend making reservations for dinner, especially at the restaurants listed, well in advance.  You can usually book online.
*   **Flexibility:** This is a suggestion. Feel free to adjust it based on your interests and energy levels.
*   **Opening Hours:** Double-check the opening hours of all attractions and restaurants before you go, as they can change.



---

**Day 1: Arrival & Harbour Exploration (August 18th - Saturday)**

*   **9:00 AM:** Arrive at Sydney Airport (SYD). Take the Airport Link train to your accommodation in the city center (e.g., Circular Quay, Darling Harbour, CBD). Check in.
*   **11:00 AM:** Settle in and take a stroll around Circular Quay. Admire the Sydney Opera House and Harbour Bridge.
*   **12:30 PM:** **Lunch:** Casual lunch at a cafe near Circular Quay. Many options available.
*   **2:00 PM:**  **Sydney Opera House Tour:** Take a guided tour of the iconic Sydney Opera House.  Allow 1.5 - 2 hours.  Book in advance!
*   **4:00 PM:** **Harbour Bridge Climb (Optional):** If you're feeling adventurous, consider a climb of the Harbour Bridge for spectacular views.  Book well in advance. Alternatively, walk across the bridge.
*   **6:00 PM:** **Dinner:** **Bennelong** (inside the Opera House). *Specialty: Sydney Rock Oysters*. Enjoy a fine dining experience with stunning harbour views.  Make a reservation!
*   **8:30 PM:**  Evening stroll along the harbour, enjoying the illuminated Opera House and Harbour Bridge.



**Day 2: Bondi Beach & Coastal Walk (August 19th - Sunday)**

*   **9:00 AM:** **Breakfast:**  Enjoy breakfast at a cafe in your neighborhood.
*   **10:00 AM:** Travel to Bondi Beach (train or bus).
*   **10:30 AM - 1:00 PM:** **Bondi Beach:** Relax on the beach, swim, or try surfing.  Consider a surf lesson.
*   **1:00 PM:** **Lunch:** Casual lunch at a cafe overlooking Bondi Beach.
*   **2:00 PM - 4:00 PM:** **Bondi to Coogee Coastal Walk:**  Enjoy the scenic coastal walk from Bondi to Coogee (approx. 6km).  Allow 2-3 hours, stopping at beaches and viewpoints along the way.
*   **4:00 PM:** **Coogee Beach:** Relax at Coogee Beach.
*   **6:00 PM:** **Dinner:**  Dinner at a restaurant in Coogee.  Many options available with ocean views.
*   **8:00 PM:** Return to your accommodation.



**Day 3:  Culture & Markets (August 20th - Monday)**

*   **9:00 AM:** **Breakfast:**  At your accommodation or a local cafe.
*   **10:00 AM - 1:00 PM:** **Art Gallery of NSW:** Explore the extensive collection of Australian and international art.
*   **1:00 PM:** **Lunch:**  Lunch at the Art Gallery's cafe or a nearby cafe in the city.
*   **2:30 PM - 5:00 PM:** **The Rocks:** Explore the historic Rocks district. Browse the shops, visit the Rocks Discovery Museum, and learn about Sydney's convict past.
*   **5:00 PM - 7:00 PM:** **Carriageworks Farmers Market (Saturday & Sunday):**  If it's a Saturday or Sunday, visit this vibrant market for local produce, artisan goods, and food stalls. *Specialty: Kangaroo Pie*.
*   **7:30 PM:** **Dinner:** Dinner at a restaurant in The Rocks.
*   **9:00 PM:** Enjoy a drink at a pub in The Rocks.



**Day 4:  Blue Mountains Day Trip (August 21st - Tuesday)**

*   **8:00 AM:** **Breakfast:**  Grab a quick breakfast before heading out.
*   **8:30 AM - 9:30 AM:** Travel to the Blue Mountains.  You can take a train from Central Station to Katoomba (approx. 2 hours).
*   **10:00 AM - 4:00 PM:** **Blue Mountains Exploration:** Explore the Blue Mountains National Park.
    *   **Scenic World:** Ride the Scenic Railway, Scenic Cableway, and Scenic Skyway for breathtaking views.
    *   **Three Sisters:**  Visit Echo Point for iconic views of the Three Sisters rock formation.
    *   **Hiking:** Choose a hiking trail based on your fitness level.  (e.g., Wentworth Falls, Govetts Leap).
*   **1:00 PM:** **Lunch:**  Have lunch at a cafe in Katoomba or a picnic lunch amidst the scenery.
*   **4:00 PM - 6:00 PM:** Travel back to Sydney.
*   **7:00 PM:** **Dinner:** Dinner in your preferred neighborhood.



**Day 5:  North Sydney & Taronga Zoo (August 22nd - Wednesday)**

*   **9:00 AM:** **Breakfast:** At your accommodation or a local cafe.
*   **10:00 AM:** Travel to North Sydney.
*   **10:30 AM - 12:30 PM:** **North Sydney Olympic Pool:** Enjoy the Olympic-sized pool with stunning harbour views. 
*   **1:00 PM:** **Lunch:** Enjoy lunch at a cafe in North Sydney.
*   **2:30 PM - 5:30 PM:** **Taronga Zoo:** Take a ferry from Circular Quay to Taronga Zoo. Explore the zoo and enjoy the harbour views.
*   **6:00 PM:** **Dinner:** Enjoy dinner at the zoo or back in the city.
*   **8:00 PM:** Return to your accommodation.



**Day 6:  Darling Harbour & Maritime Museum (August 23rd - Thursday)**

*   **9:00 AM:** **Breakfast:** At your accommodation or a local cafe.
*   **10:00 AM - 12:00 PM:** **Australian National Maritime Museum:** Explore Australia's maritime history.
*   **12:30 PM:** **Lunch:** Lunch at a restaurant in Darling Harbour.
*   **2:00 PM - 4:00 PM:** **Darling Harbour Activities:** Choose from a variety of activities in Darling Harbour:
    *   **Wild Life Sydney Zoo:** See Australian animals.
    *   **Sea Life Sydney Aquarium:** Explore marine life.
    *   **Australian Centre for Science Exploration (SciTech):** Interactive science exhibits.
*   **6:00 PM:** **Dinner:** **Vic's Meat Market**. *Specialty: BBQ Beef Brisket*. Enjoy a delicious BBQ meal.  Make a reservation!
*   **8:00 PM:** Evening stroll around Darling Harbour.



**Day 7:  South Head & Lavender Bay (August 24th - Friday)**

*   **9:00 AM:** **Breakfast:** At your accommodation or a local cafe.
*   **10:00 AM:** Travel to South Head.
*   **10:30 AM - 1:00 PM:** **South Head:** Explore the historic South Head Quarantine Station. Enjoy panoramic views of Sydney Harbour.
*   **1:00 PM:** **Lunch:** Enjoy lunch at a cafe near South Head.
*   **2:30 PM - 4:30 PM:** **Lavender Bay:** Relax at Lavender Bay, a picturesque bay with beautiful views of the city skyline.
*   **5:00 PM:** **Dinner:** **Saint Peter**. *Specialty: Murray Cod*. Indulge in a seafood-focused dinner.  Requires advance booking!
*   **7:30 PM:** Enjoy a final evening stroll and perhaps a drink at a harbour-side bar.



**Day 8: Departure (August 25th - Saturday)**

*   **9:00 AM:** **Breakfast:** Enjoy a final breakfast in Sydney.
*   **10:00 AM:** Last-minute souvenir shopping or revisit a favorite spot.
*   **12:00 PM:** Check out of your accommodation

## ساخت برنامه جامع سفر برای همه شهرها

یک حلقه فور و بعد بوممممم! همه چیز برای همه شهرها تکرار می‌شه.

In [ ]:
# دیکشنری خالی برای ثبت برنامه همه شهرها
detailed_itinerary = {}

 # حلقه تکرار برای واکشی اطلاعات همه مقصدها
for trip_stop in itinerary:
    city = trip_stop["City"]
    country = trip_stop["Country"]
    arrival = trip_stop["Arrival"]
    departure = trip_stop["Departure"]

    rest_dict = read_csv(f"{city}.csv")

    print(f"Creating detailed itinerary for {city}, {country}.")

    prompt = f"""I will visit {city}, {country} from {arrival} to {departure}.
    Create a daily itinerary with detailed activities.
    Designate times for breakfast, lunch, and dinner.

    I want to visit the restaurants listed in the restaurant dictionary without repeating any place.
    Make sure to mention the specialty that I should try at each of them.

    Restaurant dictionary:
    {rest_dict}

    """
    # ذخیره برنامه سفر پیشنهادی برای همه شهرها در دیکشنری
    detailed_itinerary[city] = print_llm_response(prompt)

Creating detailed itinerary for New York, USA.
Creating detailed itinerary for Rio de Janeiro, Brazil.
Creating detailed itinerary for Cape Town, South Africa.
Creating detailed itinerary for Istanbul, Turkey.
Creating detailed itinerary for Paris, France.
Creating detailed itinerary for Tokyo, Japan.
Creating detailed itinerary for Sydney, Australia.


حالا برنامه هر شهری که بخواهی کافیه نامش رو از دیکشنری صدا بزنی.

In [ ]:
# Print in Markdown format
print(detailed_itinerary["Tokyo"])

Okay, here's a detailed itinerary for your Tokyo trip from August 10th to August 17th, incorporating your restaurant preferences and aiming for a balanced mix of cultural experiences, food, and relaxation.  I've included estimated times, but feel free to adjust based on your pace.  August in Tokyo is hot and humid, so I've built in some breaks and considered indoor activities.

**Important Notes:**

*   **Reservations are *essential* for many of these restaurants, *especially* Sukiyabashi Jiro, Narisawa, Ginza Kojyu, and Akasaka Kikunoi.**  Book *well* in advance (weeks or even months).  I recommend using your hotel concierge or a service like TableCheck.
*   **Transportation:**  Tokyo has an excellent public transportation system (trains and subways).  Consider purchasing a Suica or Pasmo card for easy travel.
*   **Pocket Wifi/eSIM:**  Highly recommended for navigation and communication.
*   **Heat & Humidity:** Stay hydrated, wear light clothing, and be prepared for potential rain. 

## تمرین

کد پایین رو اصلاح کن تا برنامه دریافتی برای تک تک شهرهای پایین رو ببینی:

**گزینه‌ها:**
- Cape Town
- Istanbul
- New York
- Paris
- Rio de Janeiro
- Sydney
- Tokyo

In [ ]:
# نام شهر رو جایگزین کن
print(detailed_itinerary["YOUR CITY HERE"])

## شادباش! این هفته هم تموم شد 🎉🎉🎉

مرور کن. تمرین کن. فکرهات رو امتحان کن. خراب کن. گیر کن. ادامه بده. پیشرفت می‌کنی